# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=alena password=12345")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    '''Iterates over the file system, looks for json log files and appends them to a list.
    '''
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
#             all_files.append(os.path.abspath(f))
            all_files.append(os.path.relpath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]
filepath

'data/song_data/A/A/C/TRAACZK128F4243829.json'

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARGUVEV1187B98BA17,NaN,NaN,,Sierra Maestra,SOGOSOV12AF72A285E,¿Dónde va Chichi?,313.12934,1997


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOGOSOV12AF72A285E',
 '¿Dónde va Chichi?',
 'ARGUVEV1187B98BA17',
 1997,
 313.12934]

In [8]:
# create song_table
# cur.execute(song_table_create)

In [ ]:
cur.execute("create table real_json as (select value::jsonb from temp_json join lateral json_array_elements(values::json") on true)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         1 non-null      int64  
 1   artist_id         1 non-null      object 
 2   artist_latitude   0 non-null      float64
 3   artist_longitude  0 non-null      float64
 4   artist_location   1 non-null      object 
 5   artist_name       1 non-null      object 
 6   song_id           1 non-null      object 
 7   title             1 non-null      object 
 8   duration          1 non-null      float64
 9   year              1 non-null      int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 208.0+ bytes


In [11]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARGUVEV1187B98BA17', 'Sierra Maestra', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
# cur.execute(artist_table_create)

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [6]:
log_files = get_files('data/log_data')

In [7]:
filepath = log_files[0]
filepath

'data/log_data/2018/11/2018-11-22-events.json'

In [8]:
df = pd.read_json(filepath, lines=True)
df.head()  # 98

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,1542845032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,1542845350796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
2,Nirvana,Logged In,Lily,F,40,Koch,215.11791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Love Buzz,200,1542845526796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
3,Weezer,Logged In,Lily,F,41,Koch,479.32036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Only In Dreams,200,1542845741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4,Nightwish,Logged In,Lily,F,42,Koch,286.19710,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,She Is My Sin,200,1542846220796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         82 non-null     object 
 1   auth           98 non-null     object 
 2   firstName      91 non-null     object 
 3   gender         91 non-null     object 
 4   itemInSession  98 non-null     int64  
 5   lastName       91 non-null     object 
 6   length         82 non-null     float64
 7   level          98 non-null     object 
 8   location       91 non-null     object 
 9   method         98 non-null     object 
 10  page           98 non-null     object 
 11  registration   91 non-null     float64
 12  sessionId      98 non-null     int64  
 13  song           82 non-null     object 
 14  status         98 non-null     int64  
 15  ts             98 non-null     int64  
 16  userAgent      91 non-null     object 
 17  userId         98 non-null     object 
dtypes: float64(2

In [24]:
df = df[df.page == 'NextSong']
df.head()  # 82

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,1542845032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,1542845350796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
2,Nirvana,Logged In,Lily,F,40,Koch,215.11791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Love Buzz,200,1542845526796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
3,Weezer,Logged In,Lily,F,41,Koch,479.32036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Only In Dreams,200,1542845741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4,Nightwish,Logged In,Lily,F,42,Koch,286.19710,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,She Is My Sin,200,1542846220796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [30]:
df.userId.value_counts()

15     33
97     29
101     6
6       3
26      3
23      2
83      1
76      1
50      1
63      1
80      1
32      1
Name: userId, dtype: int64

In [10]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-22 00:03:52.796
1   2018-11-22 00:09:10.796
2   2018-11-22 00:12:06.796
3   2018-11-22 00:15:41.796
4   2018-11-22 00:23:40.796
Name: ts, dtype: datetime64[ns]

In [14]:
type(t[0])

pandas._libs.tslibs.timestamps.Timestamp

In [15]:
time_data = [t.dt.time, 
             t.dt.hour,
             t.dt.day,
             t.dt.isocalendar().week,
             t.dt.month,
             t.dt.year,
             t.dt.weekday]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [21]:
type(t.dt.time[0])

datetime.time

In [22]:
t.dt.weekday


0     3
1     3
2     3
3     3
4     3
     ..
89    3
90    3
91    3
92    3
95    3
Name: ts, Length: 82, dtype: int64

In [22]:
time_df = pd.DataFrame({k:v for k, v in zip(column_labels, time_data)})
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,00:03:52.796000,0,22,47,11,2018,3
1,00:09:10.796000,0,22,47,11,2018,3
2,00:12:06.796000,0,22,47,11,2018,3
3,00:15:41.796000,0,22,47,11,2018,3
4,00:23:40.796000,0,22,47,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
cur.execute(time_table_create)

In [24]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
df.level.value_counts()  # should I make them boolean?

paid    63
free    19
Name: level, dtype: int64

In [26]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head(1)

,userId,firstName,lastName,gender,level
0,15,Lily,Koch,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
cur.execute(user_table_create)

In [28]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [29]:
cur.execute(songplay_table_create)

In [31]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'),
                     row.userId, row.level, songid, artistid, row.sessionId,
                     row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [32]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.